# Nested Data Structures: Navigating MIT

Today, we'll build and modify a recursive data structure: a graph containing nodes representing courses. We maintain directed edges in the graph to represent prerequisites. We'll see that organizing the data in such a structure makes it easier to answer the questions we're interested in.

## Freshman Year
You're an excited course 6 freshman, and you have a list of courses you want to take:

In [1]:
my_courses = [
    '6.0001',
    '6.004',
    '6.009',
    '6.042',
    '6.006',
    '6.031',
    '6.170',
    '6.033'
]

You have discovered that there are ratings for the courses you're interested in:

In [2]:
my_courses_with_ratings = [
    ('6.0001', 6.8),
    ('6.004', 6.6),
    ('6.009', 7),
    ('6.042', 6.2),
    ('6.006', 6.8),
    ('6.031', 6.5),
    ('6.170', 6.3),
    ('6.033', 6.4),
]

Just as you try to register for all of your courses in one semester, you recall that some of the courses have prerequisites. You browse the course catalog and make the prereqs list here. The first element in the tuple is the course that is required for the second element.

In [3]:
prereqs = [
    ('6.0001', '6.009'),
    ('6.0001', '6.006'),
    ('6.0001', '6.004'),
    ('6.0001', '6.01'),
    ('6.009', '6.031'),
    ('6.031', '6.170'),
    ('6.004', '6.033'),
    ('6.042', '6.006'),
    ('6.006', '6.046'),
    ('6.003', '6.011'),
    ('6.0001', '6.02'),
    ('18.01', '18.02'),
    ('18.02', '18.03'),
    ('18.02', '18.06'),
    ('8.01', '8.02'),
    ('18.06', '18.065'),
    ('6.004', '6.823'),
    ('6.033', '6.824'),
    ('6.042', '6.045'),
    ('6.009', '6.033'),
    ('6.031', '6.172'),
    ('6.006', '6.172'),
    ('6.004', '6.172'),
    ('6.031', '6.170'),
    ('6.006', '6.170'),
    ('18.02', '18.600'),
    ('18.600', '18.650'),
    ('18.03', '6.003'),
    ('8.02', '6.003'),
]

Noticing that your prereq list is much longer than the list of courses you want to take, you decide to preprocess the data into a more useful form. 

In [4]:
class Course:
    def __init__(self, number, rating):
        self.number = number
        self.rating = rating
        self.prereqs = {}
        self.following = {}
        
    def get_immediate_prereqs(self):
        return list(self.prereqs.values())
    
    def get_immediate_following(self):
        return list(self.following.values())
    
    def add_prereq(self, prereq):
        self.prereqs[prereq.number] = prereq
    
    def add_following(self, following):
        self.following[following.number] = following
        
    def remove_prereq(self, prereq):
        del self.prereqs[prereq.number]
        
    def remove_following(self, following):
        del self.following[following.number]
    
    def get_all_prereqs(self):
        all_prereqs = self.prereqs.copy()
        
        # Recursively find the prereq's prereqs
        for prereq in self.prereqs.values():
            all_prereqs.update(prereq.get_all_prereqs())
            
        return all_prereqs
    
    def get_all_following(self):
        all_following = self.following.copy()
        
        # Recursively find the following's followings
        for following in self.following.values():
            all_following.update(following.get_all_following())
            
        return all_following
    
    def is_recommended(self, min_rating):
        class_is_recommended = self.rating >= min_rating
        prereqs_are_recommended = all({pr.is_recommended(min_rating) for pr in self.get_immediate_prereqs()})
        return class_is_recommended and prereqs_are_recommended

Now that you can represent a single course, you want to make a CourseRoad to help you map out your MIT career. 

In [5]:
class CourseRoad:
    def __init__(self, courses, prereqs):
        self.courses = {}
        self.intros = set()
        
        for course_number, rating in courses:
            self.courses[course_number] = Course(course_number, rating)
        
        for prereq_number, following_number in prereqs:
            if prereq_number in self.courses and following_number in self.courses:
                self.make_prereq(prereq_number, following_number)
                
        for course in self.courses.values():
            if len(course.get_immediate_prereqs()) == 0:
                self.intros.add(course.number)
        
    def make_prereq(self, prereq_number, following_number):
        prereq = self.courses[prereq_number]
        following = self.courses[following_number]
        
        prereq.add_following(following)
        following.add_prereq(prereq)
            
    def get_course_ordering(self):
        def helper(course, taken):
            course_order = []
            for following in course.get_immediate_following():
                # Take the course if all prereqs are satisfied
                if all({c.number in taken for c in following.get_immediate_prereqs()}):
                    taken.add(following.number)
                    course_order.append(following.number)
                    course_order += helper(following, taken)
            return course_order
        
        course_order = []
        taken = set()
        
        for intro_number in self.intros:
            intro_course = self.courses[intro_number]
            
            # Take the intro course
            course_order.append(intro_number)
            taken.add(intro_number)
            
            # Take the courses following the intro course
            course_order.extend(helper(intro_course, taken))
            
        return course_order
        
    def delete_course(self, course_number):
        course = self.courses[course_number]
        del self.courses[course_number]
        
        for following in course.get_immediate_following():
            following.remove_prereq(course)
            
        for prereq in course.get_immediate_prereqs():
            prereq.remove_following(course)
            
        for following in course.get_immediate_following():
            for prereq in course.get_immediate_prereqs():
                prereq.add_following(following)
                following.add_prereq(prereq)
    
    def is_course_recommended(self, course_number, min_rating):
        return self.courses[course_number].is_recommended(min_rating)
    
    def get_all_prereqs(self, course_number):
        return set(self.courses[course_number].get_all_prereqs().keys())
    
    def get_all_following(self, course_number):
        return set(self.courses[course_number].get_all_following().keys())
    
    def print_out(self):
        """
        For debugging and visualization purposes only.
        """
        for number, course in self.courses.items():
            prereq_numbers = {c.number for c in course.get_immediate_prereqs()}
            following_numbers = {c.number for c in course.get_immediate_following()}
            print(
                "{0:21}{1:5}{2:10}{3:5}{4:20}".format(
                    "" if prereq_numbers == set() else str(prereq_numbers),
                    "" if prereq_numbers == set() else "->",
                    number,
                    "" if following_numbers == set() else "->",
                    "" if following_numbers == set() else str(following_numbers),
                )
            )
    

In [6]:
my_courseroad = CourseRoad(my_courses_with_ratings, prereqs)

In [7]:
my_courseroad.print_out()

                          6.0001    ->   {'6.009', '6.006', '6.004'}
{'6.0001'}           ->   6.004     ->   {'6.033'}           
{'6.0001'}           ->   6.009     ->   {'6.033', '6.031'}  
                          6.042     ->   {'6.006'}           
{'6.0001', '6.042'}  ->   6.006     ->   {'6.170'}           
{'6.009'}            ->   6.031     ->   {'6.170'}           
{'6.006', '6.031'}   ->   6.170                              
{'6.009', '6.004'}   ->   6.033                              


## Sophomore Year 
Wanting to plan your life even more, you want a function in CourseRoad that will give you a valid ordering of the courses you want to take, so that all of the prerequisites are fulfilled for a course before you take it.

In [8]:
my_courseroad = CourseRoad(my_courses_with_ratings, prereqs)

In [9]:
my_courseroad.get_course_ordering()

['6.0001', '6.009', '6.031', '6.004', '6.033', '6.042', '6.006', '6.170']

## Junior Year
This year, you're starting to feel a bit hosed. Your friend tells you that 6.031 is a super hard course, so you decide to pretend it doesn't exist: you will not take the course. But, any prereqs for 6.031 will have to be prereqs for courses that require it. You add functionality to your CourseRoad accordingly.

In [10]:
my_courseroad = CourseRoad(my_courses_with_ratings, prereqs)

In [11]:
my_courseroad.delete_course('6.031')

In [12]:
my_courseroad.print_out()

                          6.0001    ->   {'6.009', '6.006', '6.004'}
{'6.0001'}           ->   6.004     ->   {'6.033'}           
{'6.0001'}           ->   6.009     ->   {'6.170', '6.033'}  
                          6.042     ->   {'6.006'}           
{'6.0001', '6.042'}  ->   6.006     ->   {'6.170'}           
{'6.009', '6.006'}   ->   6.170                              
{'6.009', '6.004'}   ->   6.033                              


In [13]:
my_courseroad.get_course_ordering()

['6.0001', '6.009', '6.004', '6.033', '6.042', '6.006', '6.170']

## Senior Year
Now that you've taken many classes at MIT, you'd like to know whether or not you've taken all the prereqs for a particular course. This includes both the immediate prereqs and the prereqs of those prereqs. Similarly, you'd like to know all of the courses that follow a course you've liked

In [14]:
my_courseroad = CourseRoad(my_courses_with_ratings, prereqs)

In [15]:
my_courseroad.get_all_prereqs('6.170')

{'6.0001', '6.006', '6.009', '6.031', '6.042'}

In [16]:
my_courseroad.get_all_prereqs('6.031')

{'6.0001', '6.009'}

In [17]:
my_courseroad.get_all_following('6.031')

{'6.170'}

In [18]:
my_courseroad.get_all_following('6.170')

set()

## Super Senior Year
Remembering your lovely days in 6.009, you realize that your standards are high for the courses you'll take in your extra year. You only wish to take courses above a certain rating, all of the prereqs (and prereqs of prereqs) of which are also above that rating. You add functionality to your CourseRoad to tell you if a course and all of its prereqs are above your standard.

In [19]:
my_courseroad = CourseRoad(my_courses_with_ratings, prereqs)

In [20]:
my_courseroad.is_course_recommended('6.170', 6.1)

True

In [21]:
my_courseroad.is_course_recommended('6.170', 6.5)

False

## Happy Graduation!